In [2]:
#Apache Spark : Genel amaçlı paralel veri işleme frameworku
#Hadoop üzerinde çalışabilir
#Büyük Veri İşleme, Büyük Veri ile Makine Öğrenmesi ===> HADOOP + Spark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [248]:
from pyspark import SparkContext
from datetime import datetime
from geopy.distance import geodesic
sc = SparkContext.getOrCreate()
deprem = sc.textFile("DepremVerileri.txt")
deprem.take(1)

header = deprem.first()
deprem = deprem.filter(lambda line: line != header)

def splitter(data):
  arg = data.split("\t")
  timestamp = arg[2]+'-'+arg[3].split('.')[0]
  lat = float(arg[4]);lng = float(arg[5])
  intensity = arg[7];zone = arg[14]
  try:
    date = datetime.strptime(timestamp, "%Y.%m.%d-%H:%M:%S")
  except:
    date = 'Incorrect'
  return (date,timestamp,lat,lng,intensity,zone)
depremRDD = deprem.map(lambda line :splitter(line))
depremRDD2 = depremRDD.filter(lambda x : x[0] != 'Incorrect')\
.filter(lambda x : x[0].year>= 1990 and x[0].year<=2019)
sortedRDD = depremRDD2.sortBy(lambda x : x[4], ascending=False)\
.zipWithIndex()\
.filter(lambda x: x[1]<10)
#sortedRDD.take(2)

cartesian_rdd = sortedRDD.cartesian(depremRDD2)

def distance(line):
  dep1=line[0][0];dep2=line[1];lat1 = dep1[2]
  lon1 = dep1[3];tarih1 = dep1[0];lat2 = dep2[2]
  lon2 = dep2[3];tarih2 = dep2[0]
  distance = geodesic((lat1,lon1), (lat2,lon2)).kilometers
  timezone = (tarih1-tarih2).total_seconds()/3600

  return (distance,timezone,dep1,dep2)



result = cartesian_rdd.map(lambda line : distance(line))\
.filter(lambda line: line[0]<=20 and line[1]<=24 and line[1]>=-24 and line[1] != 0)


result.collect()

[(5.087756842360975,
  -23.815,
  (datetime.datetime(2011, 10, 23, 10, 41, 21),
   '2011.10.23-10:41:21',
   38.7212,
   43.411,
   '7.2',
   'YEMLICE- (VAN) [North West  1.5 km]'),
  (datetime.datetime(2011, 10, 24, 10, 30, 15),
   '2011.10.24-10:30:15',
   38.7217,
   43.4695,
   '4.5',
   'YEMLICE- (VAN) [South East  2.4 km]')),
 (6.117905766212716,
  -23.623333333333335,
  (datetime.datetime(2011, 10, 23, 10, 41, 21),
   '2011.10.23-10:41:21',
   38.7212,
   43.411,
   '7.2',
   'YEMLICE- (VAN) [North West  1.5 km]'),
  (datetime.datetime(2011, 10, 24, 10, 18, 45),
   '2011.10.24-10:18:45',
   38.7362,
   43.3433,
   '3.7',
   'GUVENCLI- (VAN) [East 2.3 km]')),
 (7.864434478756931,
  -23.491944444444446,
  (datetime.datetime(2011, 10, 23, 10, 41, 21),
   '2011.10.23-10:41:21',
   38.7212,
   43.411,
   '7.2',
   'YEMLICE- (VAN) [North West  1.5 km]'),
  (datetime.datetime(2011, 10, 24, 10, 10, 52),
   '2011.10.24-10:10:52',
   38.7382,
   43.4988,
   '3.8',
   'AKCAOREN- (VAN) [Nor

In [221]:

result = cartesian_rdd.map(lambda line : distance(line))\
.filter(lambda line: line[0]<=20 and line[1]<=24 and line[1]>=-24 and line[1] != 0)\
.sortBy(lambda x : x[0][2][4], ascending=True)


In [241]:
a = [(1032.6521657222318,
  -178547.30805555556,
  ((1999, 8, 17, 0, 1, 37),
   '1999.08.17-00:01:37',
   40.76,
   29.97,
   '7.4',
   'BASISKELE (KOCAELI) [North East  2.0 km]'),
  ((2019, 12, 29, 11, 20, 6),
   '2019.12.29-11:20:06',
   40.3462,
   42.1595,
   '4.3',
   'GULLUDAG-NARMAN (ERZURUM) [South East  3.7 km]'))]

print(a[0][2][4])

7.4
